In [1]:
from isochrones import get_ichrone

mist = get_ichrone('mist')

In [33]:
import pandas as pd
import os
import glob
from isochrones.config import ISOCHRONES

class BolometricCorrectionGrid(object):
    
    index_cols = ('Teff', 'logg', '[Fe/H]', 'Av', 'Rv')
    
    def __init__(self, name, phot):
        self.name = name
        self.phot = phot
        
        self._df = None
    
    @property
    def directory(self):
        return os.path.join(ISOCHRONES, 'BC', self.name)
    
    def _get_filename(self, feh):
        rootdir = self.directory
        sign_str = 'm' if feh < 0 else 'p'
        filename = 'feh{0}{1:03.0f}.{2}'.format(sign_str, abs(feh)*100, self.phot)
        return os.path.join(rootdir, filename)

    def parse_table(self, filename):
        """Reads text table into dataframe
        """
        with open(filename) as fin:
            for i, line in enumerate(fin):
                if i == 5:
                    names = line[1:].split()
                    break        
        return pd.read_csv(filename, names=names, delim_whitespace=True, comment='#',
                          index_col=self.index_cols)
                    
    def get_table(self, feh):
        return self.parse_table(self._get_filename(feh))
        
    @property
    def hdf_filename(self):
        return os.path.join(self.directory, '{}.h5'.format(self.phot))
        
    def get_df(self):  #TODO: add file downloads
        if not os.path.exists(self.hdf_filename):
            filenames = glob.glob(os.path.join(self.directory, '*.{}'.format(self.phot)))
            df = pd.concat([self.parse_table(f) for f in filenames]).sort_index()
            df.to_hdf(self.hdf_filename, 'df')
        return pd.read_hdf(self.hdf_filename)
    
    @property
    def df(self):
        if self._df is None:
            self._df = self.get_df()
        return self._df
    
class MISTBolometricCorrectionGrid(BolometricCorrectionGrid):
    name = 'mist'
    
    def __init__(self, phot):
        self.phot = phot
        self._df = None
        
    def get_df(self, *args, **kwargs):
        df = super().get_df(*args, **kwargs)
        return df.xs(3.1, level='Rv')

In [34]:
bc = MISTBolometricCorrectionGrid('UBVRIplus')

In [35]:
bc.df.head()

Bessell_U  Bessell_B  Bessell_V  Bessell_R  \
Teff   logg [Fe/H] Av                                                 
2500.0 -4.0 -4.0   0.00 -11.699506  -7.714880  -4.554260  -2.464870   
                   0.05 -11.774942  -7.773749  -4.601857  -2.502038   
                   0.10 -11.850375  -7.832595  -4.649439  -2.539179   
                   0.15 -11.925804  -7.891418  -4.697008  -2.576292   
                   0.20 -12.001229  -7.950218  -4.744563  -2.613378   

                         Bessell_I   2MASS_J   2MASS_H  2MASS_Ks  Kepler_Kp  \
Teff   logg [Fe/H] Av                                                         
2500.0 -4.0 -4.0   0.00  -0.704697  1.845781  2.927064  3.436304  -2.347335   
                   0.05  -0.733802  1.831466  2.917990  3.430463  -2.379940   
                   0.10  -0.762898  1.817153  2.908916  3.424623  -2.412506   
                   0.15  -0.791985  1.802841  2.899842  3.418782  -2.445036   
                   0.20  -0.821063  1.788530  2.890769  3.412942  -2.477527   

                         Kepler_D51  Hipparcos_Hp   Tycho_B   Tycho_V  \
Teff   logg [Fe/H] Av                                                   
2500.0 -4.0 -4.0   0.00   -5.755940     -3.957971 -8.863651 -4.928598   
                   0.05   -5.809623     -3.997435 -8.928352 -4.977302   
                   0.10   -5.863305     -4.036853 -8.993041 -5.025988   
                   0.15   -5.916987     -4.076224 -9.057716 -5.074656   
                   0.20   -5.970668     -4.115548 -9.122377 -5.123306   

                         Gaia_G_DR2Rev  Gaia_BP_DR2Rev  Gaia_RP_DR2Rev  \
Teff   logg [Fe/H] Av                                                    
2500.0 -4.0 -4.0   0.00      -2.181986       -4.652544       -0.881255   
                   0.05      -2.211637       -4.697700       -0.909057   
                   0.10      -2.241240       -4.742838       -0.936829   
                   0.15      -2.270797       -4.787959       -0.964571   
                   0.20      -2.300306       -4.833062       -0.992285   

                             TESS  
Teff   logg [Fe/H] Av              
2500.0 -4.0 -4.0   0.00 -0.683335  
                   0.05 -0.709131  
                   0.10 -0.734897  
                   0.15 -0.760633  
                   0.20 -0.786338

In [36]:
from isochrones.interp import DFInterpolator

interp = DFInterpolator(bc.df, is_full=True)

In [39]:
interp((4500., 2., 0.10, 4.), 'Gaia_G_DR2Rev')

-3.0531656

In [5]:
from isochrones.interp import interp_value_4d

interp_value_4d((5704., 4.34, -2.5, 0.3), interp.grid, 2, interp.index_columns)

-1.3891660950400009

In [18]:
import numpy as np
from numba import jit
from numba import float64, boolean, uint32

from isochrones.interp import searchsorted

@jit(nopython=True)
def find_indices(point, iis):
    ndim = len(point)
    
    indices = np.zeros(ndim, dtype=uint32)
    norm_distances = np.zeros(ndim, dtype=float64)
    out_of_bounds = False
    for i in range(ndim):
        ii = iis[i]
        n = len(ii)
        x = point[i]
        ix, eq = searchsorted(ii, n, x)
        if eq:
            indices[i] = ix
            norm_distances[i] = 0
        else:
            ix = ix - 1
            indices[i] = ix
            dx = (ii[ix + 1] - ii[ix])
            norm_distances[i] = (x - ii[ix]) / dx
        out_of_bounds &= x < ii[0] or x > ii[n-1]
        
    return indices, norm_distances, out_of_bounds
                
@jit(nopython=True)
def interp_value_3d(point, grid, icol, iis):

    indices, norm_distances, out_of_bounds = find_indices(point, iis)
    
    # The following should be equivalent to 
    #  edges = np.array(list(itertools.product(*[[i, i+1] for i in indices])))
    
    ndim = 3
    nvals = 2**ndim
    edges = np.zeros((nvals, ndim))
    for i in range(nvals):
        for j in range(ndim):
            edges[i, j] = indices[j] + ((i >> (ndim - 1 - j)) & 1)  # woohoo!

    value = 0
    for j in range(nvals):
        edge_indices = np.zeros(ndim, dtype=uint32)
        for k in range(ndim):
            edge_indices[k] = edges[j, k]
            
        weight = 1.
        for ei, i, yi in zip(edge_indices, indices, norm_distances):
            if ei == i:
                weight *= 1 - yi
            else:
                weight *= yi
            
            # Now, get the value; this is why general ND doesn't work
            grid_indices = (edge_indices[0], edge_indices[1], edge_indices[2], icol)
        value += grid[grid_indices] * weight

    return value

@jit(nopython=True)
def interp_value_4d(point, grid, icol, iis):

    indices, norm_distances, out_of_bounds = find_indices(point, iis)
    
    # The following should be equivalent to 
    #  edges = np.array(list(itertools.product(*[[i, i+1] for i in indices])))
    
    ndim = 4
    nvals = 2**ndim
    edges = np.zeros((nvals, ndim))
    for i in range(nvals):
        for j in range(ndim):
            edges[i, j] = indices[j] + ((i >> (ndim - 1 - j)) & 1)  # woohoo!

    value = 0
    for j in range(nvals):
        edge_indices = np.zeros(ndim, dtype=uint32)
        for k in range(ndim):
            edge_indices[k] = edges[j, k]
            
        weight = 1.
        for ei, i, yi in zip(edge_indices, indices, norm_distances):
            if ei == i:
                weight *= 1 - yi
            else:
                weight *= yi
            
            # Now, get the value; this is why general ND doesn't work
            grid_indices = (edge_indices[0], edge_indices[1], edge_indices[2], 
                            edge_indices[3], icol)
        value += grid[grid_indices] * weight

    return value

# @jit(nopython=True)
def interp_value_5d(point, grid, icol, iis):

    indices, norm_distances, out_of_bounds = find_indices(point, iis)
    print(indices)
    # The following should be equivalent to 
    #  edges = np.array(list(itertools.product(*[[i, i+1] for i in indices])))
    
    ndim = 5
    nvals = 2**ndim
    edges = np.zeros((nvals, ndim))
    for i in range(nvals):
        for j in range(ndim):
            edges[i, j] = indices[j] + ((i >> (ndim - 1 - j)) & 1)  # woohoo!

    value = 0
    for j in range(nvals):
        edge_indices = np.zeros(ndim, dtype=int)
        for k in range(ndim):
            edge_indices[k] = edges[j, k]
            
        weight = 1.
        for ei, i, yi in zip(edge_indices, indices, norm_distances):
            if ei == i:
                weight *= 1 - yi
            else:
                weight *= yi
            
            # Now, get the value; this is why general ND doesn't work
            grid_indices = (edge_indices[0], edge_indices[1], edge_indices[2], 
                            edge_indices[3], edge_indices[4], icol)
        value += grid[grid_indices] * weight

    return value

In [20]:
%timeit interp_value_4d((5704., 4.34, -2.5, 0.3), interp.grid, 2, interp.index_columns)

The slowest run took 4.98 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 4.63 µs per loop


In [9]:
%timeit interp_value_3d((0.01, 9.23, 400.5), mist.interp.grid, 5, mist.interp.index_columns)

The slowest run took 8.53 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 3.61 µs per loop


In [23]:
from isochrones.interp import interp_value
%timeit interp_value(0.01, 9.23, 400.5, mist.interp.grid, 5, *mist.interp.index_columns)

The slowest run took 5.97 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 2.85 µs per loop


In [9]:
2**32

4294967296

In [113]:
@jit(nopython=True)
def test_index(grid, indices):
    return grid[indices]

In [114]:
indices = (8, 84, 400, 5)
test_index(mist.interp.grid, indices)

4.168065602243873

In [95]:
test_index(mist.interp.grid[0, ...], (84, 400, 5))

4.442971722114381

In [82]:
pars = (0.01, 9.23, 400.5)
logg_fn(pars)

array(3.76485045)

In [58]:
%debug logg_fn(pars)

NOTE: Enter 'c' at the ipdb>  prompt to continue execution.
> <string>(1)<module>()

ipdb> s
--Call--
> /Users/tdm/miniconda3/envs/isochrones/lib/python3.6/site-packages/scipy/interpolate/interpolate.py(2446)__call__()
   2444         self.values = values
   2445 
-> 2446     def __call__(self, xi, method=None):
   2447         """
   2448         Interpolation at coordinates

ipdb> n
> /Users/tdm/miniconda3/envs/isochrones/lib/python3.6/site-packages/scipy/interpolate/interpolate.py(2460)__call__()
   2458 
   2459         """
-> 2460         method = self.method if method is None else method
   2461         if method not in ["linear", "nearest"]:
   2462             raise ValueError("Method '%s' is not defined" % method)

ipdb> 
> /Users/tdm/miniconda3/envs/isochrones/lib/python3.6/site-packages/scipy/interpolate/interpolate.py(2461)__call__()
   2459         """
   2460         method = self.method if method is None else method
-> 2461         if method not in ["linear", "nearest"]:

ipdb> 
> /Users/tdm/miniconda3/envs/isochrones/lib/python3.6/site-packages/scipy/interpolate/interpolate.py(2477)__call__()
   2475             for i, p in enumerate(xi.T):
   2476                 if not np.logical_and(np.all(self.grid[i][0] <= p),
-> 2477                                       np.all(p <= self.grid[i][-1])):
   2478                     raise ValueError("One of the requested xi is out of bounds "
   2479                                      "in dimension %d" % i)

ipdb> 
> /Users/tdm/miniconda3/envs/isochrones/lib/python3.6/site-packages/scipy/interpolate/interpolate.py(2475)__call__()
   2473 
   2474         if self.bounds_error:
-> 2475             for i, p in enumerate(xi.T):
   2476                 if not np.logical_and(np.all(self.grid[i][0] <= p),
   2477                                       np.all(p <= self.grid[i][-1])):

ipdb> 
> /Users/tdm/miniconda3/envs/isochrones/lib/python3.6/site-packages/scipy/interpolate/interpolate.py(2476)__call__()
   2474         

ipdb> 
> /Users/tdm/miniconda3/envs/isochrones/lib/python3.6/site-packages/scipy/interpolate/interpolate.py(2485)__call__()
   2483             result = self._evaluate_linear(indices,
   2484                                            norm_distances,
-> 2485                                            out_of_bounds)
   2486         elif method == "nearest":
   2487             result = self._evaluate_nearest(indices,

ipdb> 
--Call--
> /Users/tdm/miniconda3/envs/isochrones/lib/python3.6/site-packages/scipy/interpolate/interpolate.py(2495)_evaluate_linear()
   2493         return result.reshape(xi_shape[:-1] + self.values.shape[ndim:])
   2494 
-> 2495     def _evaluate_linear(self, indices, norm_distances, out_of_bounds):
   2496         # slice for broadcasting over trailing dimensions in self.values
   2497         vslice = (slice(None),) + (None,)*(self.values.ndim - len(indices))

ipdb> n
> /Users/tdm/miniconda3/envs/isochrones/lib/python3.6/site-packages/scipy/interpolate/interpola

ipdb> print(weight)
1.0
ipdb> n
> /Users/tdm/miniconda3/envs/isochrones/lib/python3.6/site-packages/scipy/interpolate/interpolate.py(2505)_evaluate_linear()
   2503         for edge_indices in edges:
   2504             weight = 1.
-> 2505             for ei, i, yi in zip(edge_indices, indices, norm_distances):
   2506                 weight *= np.where(ei == i, 1 - yi, yi)
   2507             values += np.asarray(self.values[edge_indices]) * weight[vslice]

ipdb> print(weight)
[0.96]
ipdb> n
> /Users/tdm/miniconda3/envs/isochrones/lib/python3.6/site-packages/scipy/interpolate/interpolate.py(2506)_evaluate_linear()
   2504             weight = 1.
   2505             for ei, i, yi in zip(edge_indices, indices, norm_distances):
-> 2506                 weight *= np.where(ei == i, 1 - yi, yi)
   2507             values += np.asarray(self.values[edge_indices]) * weight[vslice]
   2508         return values

ipdb> 
> /Users/tdm/miniconda3/envs/isochrones/lib/python3.6/site-packages/scipy/int

ipdb> print(values
*** SyntaxError: unexpected EOF while parsing
ipdb> print(values)
[0.71941052]
ipdb> n
> /Users/tdm/miniconda3/envs/isochrones/lib/python3.6/site-packages/scipy/interpolate/interpolate.py(2504)_evaluate_linear()
   2502         values = 0.
   2503         for edge_indices in edges:
-> 2504             weight = 1.
   2505             for ei, i, yi in zip(edge_indices, indices, norm_distances):
   2506                 weight *= np.where(ei == i, 1 - yi, yi)

ipdb> 
> /Users/tdm/miniconda3/envs/isochrones/lib/python3.6/site-packages/scipy/interpolate/interpolate.py(2505)_evaluate_linear()
   2503         for edge_indices in edges:
   2504             weight = 1.
-> 2505             for ei, i, yi in zip(edge_indices, indices, norm_distances):
   2506                 weight *= np.where(ei == i, 1 - yi, yi)
   2507             values += np.asarray(self.values[edge_indices]) * weight[vslice]

ipdb> 
> /Users/tdm/miniconda3/envs/isochrones/lib/python3.6/site-packages/scipy/i

ipdb> 
> /Users/tdm/miniconda3/envs/isochrones/lib/python3.6/site-packages/scipy/interpolate/interpolate.py(2507)_evaluate_linear()
   2505             for ei, i, yi in zip(edge_indices, indices, norm_distances):
   2506                 weight *= np.where(ei == i, 1 - yi, yi)
-> 2507             values += np.asarray(self.values[edge_indices]) * weight[vslice]
   2508         return values
   2509 

ipdb> 
> /Users/tdm/miniconda3/envs/isochrones/lib/python3.6/site-packages/scipy/interpolate/interpolate.py(2503)_evaluate_linear()
   2501         edges = itertools.product(*[[i, i + 1] for i in indices])
   2502         values = 0.
-> 2503         for edge_indices in edges:
   2504             weight = 1.
   2505             for ei, i, yi in zip(edge_indices, indices, norm_distances):

ipdb> print(values)
[1.43934519]
ipdb> print(edges)
ipdb> print(list(edges))
[(array([12]), array([85]), array([400])), (array([12]), array([85]), array([401])), (array([13]), array([84]), array([400])), (ar

In [176]:
indices, norm_distances, out_of_bounds = find_indices((0.01, 9.23, 400.5), mist.interp.index_columns)

In [198]:
import itertools

edges = np.array(list(itertools.product(*[[i, i + 1] for i in indices])))

In [199]:
edges

array([[ 12,  84, 400],
       [ 12,  84, 401],
       [ 12,  85, 400],
       [ 12,  85, 401],
       [ 13,  84, 400],
       [ 13,  84, 401],
       [ 13,  85, 400],
       [ 13,  85, 401]])

In [180]:
indices

array([ 12,  84, 400], dtype=uint32)

In [179]:
list(edges)

[(12, 84, 400),
 (12, 84, 401),
 (12, 85, 400),
 (12, 85, 401),
 (13, 84, 400),
 (13, 84, 401),
 (13, 85, 400),
 (13, 85, 401)]

In [191]:
ndim = 3
nvals = 2**ndim
edges2 = np.zeros((nvals, ndim))
for i in range(nvals):
    for j in range(ndim):
        edges2[i, j] = indices[j] + ((i >> (ndim - 1 - j)) & 1)
edges2

array([[ 12.,  84., 400.],
       [ 12.,  84., 401.],
       [ 12.,  85., 400.],
       [ 12.,  85., 401.],
       [ 13.,  84., 400.],
       [ 13.,  84., 401.],
       [ 13.,  85., 400.],
       [ 13.,  85., 401.]])

In [136]:
interp_value((5770., 4.42, -2.5, 0.32, 3.1), interp.grid, 0, interp.index_columns)

(0, 0, 14, 0)
(0, 1, 15, 0)
(0, 2, 0, 0)
(0, 3, 6, 0)
(0, 4, 0, 0)
(1, 0, 13, -1)
(1, 1, 15, 0)
(1, 2, 0, 0)
(1, 3, 6, 0)
(1, 4, 0, 0)
(2, 0, 14, 0)
(2, 1, 14, -1)
(2, 2, 0, 0)
(2, 3, 6, 0)
(2, 4, 0, 0)
(3, 0, 13, -1)
(3, 1, 14, -1)
(3, 2, 0, 0)
(3, 3, 6, 0)
(3, 4, 0, 0)
(4, 0, 14, 0)
(4, 1, 15, 0)
(4, 2, 0, 0)
(4, 3, 6, 0)
(4, 4, 0, 0)


In [70]:
mist.interp.grid.shape

(15, 107, 1711, 27)